In [1]:
import datetime
import re
from pathlib import Path

from IPython.display import Markdown as md
from mini_transcript_search import Criteria, ModelHandler


def dmd(text):
    display(md(text))


top_level = Path.cwd()
while (top_level / "pyproject.toml").exists() is False:
    top_level = top_level.parent

handler = ModelHandler(
    use_local_model=False,
    override_stored=False,
    storage_path=Path(top_level, "data", "parlparse_xmls"),
    silent=True,
)

search_criteria = Criteria(
    [
        "register of members interests",
        "May I draw attention to my interests as registered in the Register of Members Financial Interests",
        "May I draw attention to my entry in the Register of Members’ Financial Interests?",
        "May I draw attention to my interests in register?",
        "May I draw attention to my interests as declared in the register?",
        "I refer Members to my registered interest.",
    ],
    score_type="nearest",
)

today = datetime.date.today()
one_month_ago = today - datetime.timedelta(days=30)


# the last week starting yesterday
last_month = ModelHandler.DateRange(start_date=one_month_ago, end_date=today)
results = handler.query(
    search_criteria,
    threshold=0.35,
    date_range=last_month,
    chamber=ModelHandler.Chamber.COMMONS,
    transcript_type=ModelHandler.TranscriptType.DEBATES,
)

# dump csv
df = results.df()

# Vector search: register of interests declarations



In [2]:
dmd(f"Last run date: {today}")

Last run date: 2024-10-09

In [3]:
def bold_icase(text: str, word: str) -> str:
    """
    Bold a word in a case insensitive way.
    We are passing 'interest' but want to bold 'Interest' for example.
    """
    return re.sub(re.compile(f"({word})", re.IGNORECASE), r"**\1**", text)

In [6]:
# atched_text	speaker_name	person_id	chamber	transcript_type	speech_id	twfy_ur
df["date"] = df["speech_id"].apply(lambda x: x.split("/")[2][:10])
df["int_person_id"] = df["person_id"].apply(lambda x: x.split("/")[-1] if x else "")

must_contain = ["declare", "interest", "register"]

passes_text = df["matched_text"].apply(
    lambda text: any([word in text.lower() for word in must_contain])
)

df = df[passes_text]

# dedupe on debate_url
df = df.drop_duplicates(subset=["debate_url"])


def bold_text(text: str) -> str:
    bolded_text = text
    for m in must_contain:
        bolded_text = bold_icase(bolded_text, m)
    return bolded_text


df["matched_text"] = df["matched_text"].apply(bold_text)

# sort by reverse date
df = df.sort_values("date", ascending=False)

for date, gdf in df.groupby("date", sort=False):
    gdf = gdf.sort_values("distance", ascending=True)
    dmd(f"# {date}")
    for _, row in gdf.iterrows():
        name_and_id = f"{row.speaker_name} ({row.int_person_id})"
        name_and_id_slug = name_and_id.replace(" ", "-").replace("(", "\\(").replace(")", "\\)")
        dmd(f"## {name_and_id}")
        dmd(f"[🔗](#{name_and_id_slug})")
        dmd(
            f"[Debate link]({row.debate_url}) | [Register link](https://www.theyworkforyou.com/mp/{row.int_person_id}/register)"
        )

        dmd(f"match score: {round(1 - row.distance,2)}")

        dmd(">" + row.matched_text)

# 2024-10-08

James-Frith-\(25622\)


## James Frith (25622)

[🔗](#James-Frith-\(25622\))

[Debate link](https://www.theyworkforyou.com/debates/?id=2024-10-08a.186.2) | [Register link](https://www.theyworkforyou.com/mp/25622/register)

...

## Jonathan Reynolds (24929)

[🔗](#Jonathan-Reynolds-\(24929\))

[Debate link](https://www.theyworkforyou.com/debates/?id=2024-09-09b.590.4) | [Register link](https://www.theyworkforyou.com/mp/24929/register)

match score: 0.65

>The hon. Gentleman will have heard my answers. He would surely agree that to get things wrong with these cases would be a terrible problem. Some of the errors and the problems that have been caused have been a frustration, but I want to assure him. I think it is better, in matters relating to this scandal, that it is the Department for Business and Trade, given the ******interest******s of Members, that provides the updates on these matters. However, I can assure him that work is proceeding at pace, and people can proactively ******register****** their credentials to ensure that there is no delay if they are in a position where they know they are one of the wronged parties.